<a href="https://colab.research.google.com/github/vaibhavkhute2212/Self-Suggestion-Tool-for-Long-Term-Investment-in-Financial-Markets/blob/main/Automation_Trading_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import math

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 637 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
data = yf.download(tickers='ETH-USD',period='1d',interval='1m')
data['Middle Band'] = data["Close"].rolling(window=21).mean() 
data['Upper Band'] =data['Middle Band'] + 2*data['Close'].rolling(window=21).std() 
data['Lower Band']= data['Middle Band'] - 2*data['Close'].rolling(window=21).std()
#Create a new data frame
period=21
new_data = data[period-1:]
#Show the new data frame
new_data

# Create a function to get the buy and sell signals
def get_signal(data):
  buy_signal=[] #buy list
  sell_signal=[] #sell list
  for i in range(len(data['Close'])):
    if data['Close'][i] > data['Upper Band'][i]:
      buy_signal.append(np.nan)
      sell_signal.append(data['Close'][i])
    elif data['Close'][i] < data['Lower Band'][i]: #Then you should buy
      #print('BUY')
      sell_signal.append(np.nan)
      buy_signal.append(data['Close'][i])
    else:
      buy_signal.append(np.nan)
      sell_signal.append(np.nan)
  return (buy_signal, sell_signal)

#Create new columns for the buy and sell signals
new_data['Buy'] =  get_signal(new_data)[0]
new_data['Sell'] =  get_signal(new_data)[1]
#ax.scatter(x_axis, new_df['Buy'] , color='green', lw=3, label = 'Buy',marker = '^', alpha = 1)
#ax.scatter(x_axis, new_df['Sell'] , color='red', lw=3, label = 'Sell',marker = 'v', alpha = 1)\


fig=go.Figure()

fig.add_trace(go.Scatter(x=new_data.index, y=new_data['Middle Band'], line=dict(color='blue', width=.7), name='Middle Band'))
fig.add_trace(go.Scatter(x=new_data.index, y=new_data['Upper Band'],line=dict(color='red', width=1.5), name='Upper Band'))
fig.add_trace(go.Scatter(x=new_data.index, y=new_data['Lower Band'],line=dict(color='green', width=1.5), name= 'Lower Band'))
fig.add_trace(go.Scatter(mode='markers',x=new_data.index, y=new_data['Buy'],marker=dict(color='Black',size=10,line=dict(color='MediumPurple',width=2)),showlegend=False))
fig.add_trace(go.Scatter(mode='markers',x=new_data.index, y=new_data['Sell'],marker=dict(color='Yellow',size=10,line=dict(color='MediumPurple',width=2)),showlegend=False))


fig.add_trace(go.Candlestick(x=new_data.index, open=new_data['Open'], high=new_data['High'],low=new_data['Low'],close=new_data['Close'], name='market data'))

fig.update_layout(title= 'Reliance Live share price evolution', yaxis_title='Stock Price (INR per Shares)')


fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(buttons=list([dict(count=15, label='15m',step = 'minute', stepmode="backward"), 
                                                                            dict(count=45, label='45m', step="minute", stepmode="backward"),
                                                                            dict(count=1, label='HTD', step="hour", stepmode="todate"), 
                                                                            dict(count=3, label="3h", step="hour", stepmode="backward"), dict(step="all")])))

fig.show()

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ETH-USD: No data found for this date range, symbol may be delisted


In [3]:
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib as ta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4023    0  4023    0     0  21398      0 --:--:-- --:--:-- --:--:-- 21513
100  503k  100  503k    0     0   575k      0 --:--:-- --:--:-- --:--:--  921k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4051    0  4051    0     0  23690      0 --:--:-- --:--:-- --:--:-- 23690
100  406k  100  406k    0     0   471k      0 --:--:-- --:--:-- --:--:--  660k


In [4]:
df = yf.download('RELIANCE.NS', period = '1d', interval = '1m')

[*********************100%***********************]  1 of 1 completed


In [5]:
df = df.drop(df[df['Volume'] == 0].index)
n=10
df['RSI'] = ta.RSI(np.array(df['Close'].shift(1)),timeperiod= n)
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3])

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = 'market data'),
              row=1, col=1)

fig.update_xaxes(
    rangeslider_visible=False)

fig.add_trace(go.Scatter(x=df.index,
                y=df['RSI'] , name = 'RSI', line=dict(color='royalblue', width=1.2)),
              row=2, col=1)

fig.show()

In [6]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)
    
    
atr_period = 7             #standard atr_period = 10
atr_multiplier = 2.5       #standard arr_multiplier = 3



df = yf.download(tickers='RELIANCE.NS',period='1d',interval='1m')
supertrend = Supertrend(df, atr_period, atr_multiplier)
df = df.join(supertrend)

import plotly.graph_objs as go

fig=go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['Final Lowerband'], line=dict(color='black', width=2), name='Final Lowerband'))
fig.add_trace(go.Scatter(x=df.index, y=df['Final Upperband'],line=dict(color='blue', width=2), name='Final Upperband'))
fig.add_trace(go.Scatter(x=df.index, y=df['Close'],line=dict(color='blue', width=0.7), name= 'Close Price'))

fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'],low=df['Low'],close=df['Close'], name='market data'))

fig.update_layout(title= 'Reliance Live share price evolution', yaxis_title='Stock Price (INR per Shares)')


fig.update_xaxes(rangeslider_visible=True, rangeselector=dict(buttons=list([dict(count=15, label='15m',step = 'minute', stepmode="backward"), 
                                                                            dict(count=45, label='45m', step="minute", stepmode="backward"),
                                                                            dict(count=1, label='HTD', step="hour", stepmode="todate"), 
                                                                            dict(count=3, label="3h", step="hour", stepmode="backward"), dict(step="all")])))

fig.show()

[*********************100%***********************]  1 of 1 completed


In [ ]:
df = yf.download(tickers='RELIANCE.NS',period='1d',interval='1m')


df['macd'], df['macdsig'], df['macdhist'] = ta.MACD(np.asarray(df['Close']),
                                                      fastperiod=14, slowperiod=23, signalperiod=10)

from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, row_heights=[0.7, 0.3])

fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'], name = 'market data'),
              row=1, col=1)

fig.update_xaxes(
    rangeslider_visible=False)

fig.add_trace(go.Scatter(x=df.index,
                y=df['macd'] , name = 'macd', line=dict(color='black', width=1.2)),
              row=2, col=1)
fig.add_trace(go.Scatter(x=df.index,
                y=df['macdsig'] , name = 'macdsig', line=dict(color='blue', width=1.2)),
              row=2, col=1)

fig.add_trace(go.Scatter(x=df.index,
                y=df['macdhist'] , name = 'macdhist', mode='lines+markers', line=dict(color='green', width=0.01)),
              row=2, col=1)

fig.show()

[*********************100%***********************]  1 of 1 completed
